# Deceleration example

In [ ]:
from cosapp.drivers import RungeKutta, NonLinearSolver
from cosapp.recorders import DataFrameRecorder
import amad.disciplines.aerodynamics.tools.createAeroInterpolationCSV as aeroInterp  # Tool to create the function to Interpolate.
from amad.disciplines.performance.systems.Deceleration import Decelerate
import numpy as np

In [ ]:
"""Aerodynamic inputs"""
# Directory to the Aero results in CSV in order to build the interpolation functions.
Aero_CSV = r'../../disciplines/aerodynamics/tools/Results/aero_results.csv'

# Ranges to create CL and CD interpolation functions from AVL Aero Results (CSV file).
# REMARK: To maintain units, array and split format to the values must correlate to the CSV file results.
alpha_list = list(np.arange(-6.0, 10.0, 2))  # unit='deg', desc='Range of AOA to create functions')
mach_list = list(np.arange(0.0, 0.84, 0.04))  # unit='', desc='Range of Mach to create functions')
altitude_list = list(np.arange(0.0, 13000, 500))  # unit='m', desc='Range of altitude to create functions')

# Creation of functions for the aerodynamic coefficients interpolation.
CLAeroIt = aeroInterp.CL_Interpolation_function(alpha_list, mach_list, altitude_list, Aero_CSV)
CDAeroIt = aeroInterp.CD_Interpolation_function(alpha_list, mach_list, altitude_list, Aero_CSV)
DAeroIt = aeroInterp.Drag_Interpolation_function(alpha_list, mach_list, altitude_list, Aero_CSV)

s1 = Decelerate(name='s1')
s1.CLAeroIt = CLAeroIt
s1.CDAeroIt = CDAeroIt
s1.DAeroIt = DAeroIt
###
driver = s1.add_driver(RungeKutta())
solver = driver.add_child(NonLinearSolver('solver'))
driver.time_interval = (0, 10000)
driver.dt = 10
Data_to_record = [
    'Altitude', 'CD', 'CL',
    'Distance', 'Drag', 'Lift',
    'Mach', 'RC', 'TAS', 'CAS',
    'THR', 'alpha', 'gamma',
    'mass', 'out_p.fuel_mass', 'time']

# Add a recorder to capture time evolution in a dataframe
driver.add_recorder(
    DataFrameRecorder(
        includes = Data_to_record,
    ), period=20
)

# Initial conditions
p_0 = np.array([0.0, 0.0, 12000.0])  # Position [m]
CS_0 = np.array([0.0])  # Fuel consumption [kg/s]
v_0 = np.array([230.0, 0.0, 6.0])  # TAS
m_0 = np.array([68000])  # mass

# Define a simulation scenario
driver.set_scenario(
    init = {'CS': CS_0, 'in_p.position': p_0, 'in_p.TAS_speed': v_0, 'm0': m_0},
    values = {
        'g': 9.81,  # [m/s^2]
        'S': 124,  # [m^2]
        'n_eng': 2,
        'Iso_Mach': 0.75
    },
)

s1.run_drivers()
data = driver.recorder.export_data()
